# Atividade 3 - Árvores de decisão e Floresta Aletatótia

**Disciplina:** Inteligência Artificial

**Professor:** Anderson Cavalcanti

**Discentes:** **Maria Clara da Silva Ferreria** & **Yann Keven Jordão Leão**

## Parte 1: Aplicando a Floresta ao Dataset Iris

### Preparando o dataset

In [ ]:
# Importantos as dependências
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# Carregando o dataset
iris = load_iris()

In [ ]:
# Definindo parâmetros(X) e Alvo(y)
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target).map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})

In [ ]:
# Separando em dados de teste e treino
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.3, random_state=64)

### Treinando o modelo de Árvore de Decisão

In [ ]:
# Criação
arvore = DecisionTreeClassifier(random_state=64)

# Teste
arvore.fit(X_train, y_train)

In [ ]:
# Fazendo previsão com o 'test'
pred_arvore = arvore.predict(X_test)

In [ ]:
# Avaliando a acurácia
acuracy_arvore = accuracy_score(y_test, pred_arvore)
acuracy_arvore

### Treinando o modelo de Floresta Aleatória

In [ ]:
# Criação
floresta = RandomForestClassifier(n_estimators=100, random_state=64)

# Teste
floresta.fit(X_train, y_train)

In [ ]:
# Fazendo previsão com o 'test'
pred_floresta = floresta.predict(X_test)

In [ ]:
# Avaliando a acurácia
acuracy_floresta = accuracy_score(y_test, pred_floresta)
acuracy_floresta

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Comparação de Desempenho no Conjunto de TESTE', fontsize=16)

# Matriz para a Árvore de Decisão
cm_arvore = confusion_matrix(y_test, pred_arvore)
sns.heatmap(cm_arvore, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=floresta.classes_, yticklabels=floresta.classes_)
axes[0].set_title('Única Árvore de Decisão')
axes[0].set_xlabel('Previsão')
axes[0].set_ylabel('Real')

# Matriz para a Floresta Aleatória
cm_floresta = confusion_matrix(y_test, pred_floresta)
sns.heatmap(cm_floresta, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=floresta.classes_, yticklabels=floresta.classes_)
axes[1].set_title('Floresta Aleatória')
axes[1].set_xlabel('Previsão')
axes[1].set_ylabel('Real')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### Analisando a importânica das Features

In [ ]:
# Uttilizando a floresta aleatória para a análise
importancias = pd.Series(data=floresta.feature_importances_, index=X.columns)

In [ ]:
# Ordenando e pegando as 10 mais importantes
importancias_sorted = importancias.sort_values(ascending=False).head(10)

In [ ]:
# Criando o gráfico
plt.figure(figsize=(10, 8))
sns.barplot(
    x=importancias_sorted,
    y=importancias_sorted.index,
    hue=importancias_sorted.index,
    palette='viridis',
    legend=False
    )
plt.title('Top 10 Features Mais Importantes', fontsize=16)
plt.xlabel('Nível de Importância')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

### Perguntas - Parte 1

**1. A única Árvore de Decisão sofreu overfitting neste novo dataset?**

Sim. O modelo de Árvore de Decisão apresentou overfitting, pois não foi definida uma profundidade máxima durante o treinamento. Isso faz com que a árvore cresça até o limite, ajustando-se completamente aos dados de treino — inclusive aos ruídos — o que leva a uma generalização ruim.
Essa situação fica evidente ao observar que o modelo obteve 100% de acurácia no conjunto de treino, enquanto a acurácia no conjunto de teste foi inferior, indicando que ele memorizou o conjunto de treino ao invés de aprender padrões gerais.

**2. A Floresta Aleatória teve um desempenho superior ao da única árvore no conjunto de teste?**

Sim. A Floresta Aleatória apresentou um desempenho melhor no conjunto de teste.
A acurácia obtida pela Floresta Aleatória foi 0.9333, enquanto a da Árvore de Decisão única foi 0.9143.
Isso representa uma diferença de aproximadamente 2,09% de desempenho superior da Floresta Aleatória.
Esse resultado é esperado, pois o método de ensemble combina diversas árvores, reduzindo a variância e, consequentemente, o risco de overfitting.

**3. De acordo com o gráfico gerado, qual foi a característica mais importante que o modelo usou para classificar as espécies de flores?**

De acordo com o gráfico de importância das variáveis, as características mais relevantes para a classificação das espécies de flores foram:

Comprimento da pétala (petal length) – a variável mais importante para a decisão do modelo;

Largura da pétala (petal width) – a segunda mais expressiva.

Já as medidas relacionadas à sépala (comprimento e largura da sépala) tiveram importância significativamente menor, indicando que as dimensões das pétalas são os principais atributos utilizados pelo modelo para distinguir as espécies de flores.

## Parte 2: Ajustando os Hiperparâmetros da Floresta

### Preparando o dataset

In [ ]:
# Carregando dataset
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target).map({0: 'Maligno', 1: 'Benigno'})

# Dividindo em teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=64)

### Tarefa 2.1: O Efeito do Número de Árvores (n_estimators)

In [ ]:
# Lista de valores para n_estimators
n_estimators_list = [1, 10, 20, 50, 100, 200]
results_estimator = []

# Loop para testar diferentes quantidades de árvores
for n in n_estimators_list:
    model = RandomForestClassifier(n_estimators=n, random_state=64)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results_estimator.append({'n_estimators': n, 'acuracia': acc})

results_df = pd.DataFrame(results_estimator)

In [ ]:
# Visualizando os resultados
plt.figure(figsize=(8, 5))
sns.lineplot(data=results_df, x='n_estimators', y='acuracia', marker='o', color='tab:blue')
plt.title('Efeito do Número de Árvores na Acurácia da Random Forest', fontsize=13)
plt.xlabel('Número de Árvores (n_estimators)')
plt.ylabel('Acurácia no Conjunto de Teste')
plt.xticks(n_estimators_list)
plt.ylim(0.9, 1.0)
plt.show()

**4. O que acontece com a acurácia à medida que aumentamos o número de árvores na
floresta? A performance melhora indefinidamente ou chega a um ponto em que o ganho é
muito pequeno?**

Ao aumentar o número de árvores na Floresta Aleatória, observamos que a acurácia melhora significativamente entre 1 e 10 árvores, passando de aproximadamente 0.918 para 0.959 — um salto considerável que demonstra o efeito positivo do ensemble (a combinação de múltiplas árvores reduz a variância e melhora a generalização do modelo).

A partir de 10 árvores, entretanto, o desempenho se estabiliza, com valores praticamente idênticos de acurácia para 10, 20, 50 e 200 árvores (em torno de 0.959).
Isso indica que, após certo ponto, adicionar mais árvores traz ganhos marginais ou nulos, pois o modelo já atingiu um nível ótimo de generalização.

Curiosamente, há uma pequena queda em 100 árvores (para 0.953), que provavelmente se deve a variações aleatórias no processo de amostragem dos dados.

### Tarefa 2.2: O Efeito da Profundidade das Árvores (max_depth)

In [ ]:
# Lista de valores para max_depth
max_depth_list = [1, 2, 3, 5, 10, None]
results = []

# Loop para testar diferentes profundidades
for n in max_depth_list:
    model = RandomForestClassifier(n_estimators=100, max_depth=n, random_state=64)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append({'max_depth': n, 'acuracia': acc})

results_depth_df = pd.DataFrame(results)

In [ ]:
# Tirando o 'None' para plotar o gráfico
results_depth_plot = results_depth_df.dropna(subset=['max_depth'])

# Visualizando os resultados
plt.figure(figsize=(8, 5))
sns.lineplot(data=results_depth_plot, x='max_depth', y='acuracia', marker='o', color='tab:orange')
plt.title('Efeito da Profundidade Máxima na Acurácia da Random Forest', fontsize=13)
plt.xlabel('Profundidade da árvore (max_depth)')
plt.ylabel('Acurácia no Conjunto de Teste')
plt.ylim(0.9, 1.0)
plt.show()

**5. Como o parâmetro max_depth afeta o overfitting? Descreva o que aconteceu com a
acurácia de treino e a de teste à medida que a profundidade das árvores aumentou.**

Ao analisar o efeito da profundidade máxima (max_depth) das árvores na Floresta Aleatória, observamos que a acurácia aumenta gradualmente até certo ponto e, a partir daí, se estabiliza.

Nos primeiros valores, há um ganho visível:

- Com profundidade 1, a acurácia é de 0.9239,
- Com profundidade 2 e 3, ela sobe para 0.9415,
- E atinge 0.9532 a partir da profundidade 5.

A partir desse ponto, mesmo ampliando a profundidade para 10 ou deixando sem limite (None), o desempenho permanece praticamente o mesmo, indicando que o modelo já capturou toda a complexidade relevante dos dados.

Esse comportamento mostra que, após uma profundidade intermediária, o modelo não ganha mais capacidade de generalização — ele apenas se torna mais complexo, o que pode até aumentar o risco de overfitting em outros conjuntos de dados.